In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.applications.resnet_v2 import ResNet50V2
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras.utils import to_categorical
import time
import tensorflow as tf

In [2]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.config.list_physical_devices('GPU')

Num GPUs Available:  1


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 1GB of memory on the first GPU
  try:
    tf.config.set_logical_device_configuration(
        gpus[0],
        [tf.config.LogicalDeviceConfiguration(memory_limit=3072)])
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
# Function to load preprocessed images and labels
def load_data(input_dirs):
    X = []
    y = []
    for i, input_dir in enumerate(input_dirs):
        for img_name in os.listdir(input_dir):
            img_path = os.path.join(input_dir, img_name)
            img = cv2.imread(img_path)
            X.append(img)
            y.append(i)
    return np.array(X), np.array(y)

In [5]:
# Data directories for preprocessed images
preprocessed_normal_folder = "new_preprocessed_clahe/normal/"
preprocessed_polyp_folder = "new_preprocessed_clahe/polyps/"
preprocessed_esophagitis_folder = "new_preprocessed_clahe/esophagitis/"
preprocessed_ulcerative_colitis_folder = "new_preprocessed_clahe/ulcerative-colitis/"

preprocessed_dirs = [preprocessed_normal_folder, preprocessed_polyp_folder, preprocessed_esophagitis_folder, preprocessed_ulcerative_colitis_folder]

# Load preprocessed images and labels
X, y = load_data(preprocessed_dirs)

# Data preprocessing
X = X.astype('float32') / 255.0
y = to_categorical(y, num_classes=4)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Loading the pre-trained InceptionV3 model
base_model = ResNet50V2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Adding custom dense layers for multi-class classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)  # Output layer with 4 units for 4 classes

# Combining the base model with custom dense layers
model = Model(inputs=base_model.input, outputs=predictions)

# Freezing layers of the base model
for layer in base_model.layers:
    layer.trainable = False

# Compiling the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
################################################################

with tf.device('/cpu:0'):
   X_train = tf.convert_to_tensor(X_train, np.float32)
   y_train = tf.convert_to_tensor(y_train, np.float32)

################################################################

In [9]:
# Start time
start_time = time.time()

# Training the model
model.fit(X_train, y_train, epochs = 10, batch_size = 8, validation_data = (X_test, y_test))

# End time
end_time = time.time()

# Duration of training
training_duration = end_time - start_time
print(f"Training took {round(training_duration, 2)} seconds.")
training_time_minutes = training_duration/60
training_time_hours = training_time_minutes/60
print(f"Training took {round(training_time_minutes, 2)} minutes.")
print(f"Training took {round(training_time_hours, 2)} hours.")

Epoch 1/10
350/350 [==============================] - 28s 80ms/step - loss: 0.0581 - accuracy: 0.9764 - val_loss: 0.3334 - val_accuracy: 0.9000
Epoch 2/10
350/350 [==============================] - 28s 80ms/step - loss: 0.0679 - accuracy: 0.9739 - val_loss: 0.3008 - val_accuracy: 0.9214
Epoch 3/10
350/350 [==============================] - 29s 82ms/step - loss: 0.0325 - accuracy: 0.9886 - val_loss: 0.2559 - val_accuracy: 0.9357
Epoch 4/10
350/350 [==============================] - 28s 80ms/step - loss: 0.0234 - accuracy: 0.9911 - val_loss: 0.2819 - val_accuracy: 0.9200
Epoch 5/10
350/350 [==============================] - 28s 80ms/step - loss: 0.0094 - accuracy: 0.9971 - val_loss: 0.3503 - val_accuracy: 0.9200
Epoch 6/10
350/350 [==============================] - 28s 81ms/step - loss: 0.0379 - accuracy: 0.9825 - val_loss: 0.3773 - val_accuracy: 0.9014
Epoch 7/10
350/350 [==============================] - 29s 82ms/step - loss: 0.1056 - accuracy: 0.9654 - val_loss: 0.3767 - val_accuracy:

In [1]:
# Model evaluation
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

NameError: name 'model' is not defined

In [ ]:
# Saving the model
model.save('saved_models/v3_resnet50v2_clahe/')

In [ ]:
from keras.models import load_model

# Load the saved model
loaded_model = load_model('saved_models/v3_resnet50v2_clahe/')

# Model evaluation
loss, accuracy = loaded_model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

In [ ]:
Epoch 1/10
350/350 [==============================] - 41s 88ms/step - loss: 0.5436 - accuracy: 0.8189 - val_loss: 0.3361 - val_accuracy: 0.8614
Epoch 2/10
350/350 [==============================] - 28s 80ms/step - loss: 0.2632 - accuracy: 0.9004 - val_loss: 0.2881 - val_accuracy: 0.8857
Epoch 3/10
350/350 [==============================] - 28s 81ms/step - loss: 0.2046 - accuracy: 0.9107 - val_loss: 0.1952 - val_accuracy: 0.9171
Epoch 4/10
350/350 [==============================] - 28s 79ms/step - loss: 0.1615 - accuracy: 0.9371 - val_loss: 0.2100 - val_accuracy: 0.9157
Epoch 5/10
350/350 [==============================] - 29s 81ms/step - loss: 0.1421 - accuracy: 0.9389 - val_loss: 0.3979 - val_accuracy: 0.8757
Epoch 6/10
350/350 [==============================] - 28s 79ms/step - loss: 0.1391 - accuracy: 0.9439 - val_loss: 0.2243 - val_accuracy: 0.9286
Epoch 7/10
350/350 [==============================] - 28s 80ms/step - loss: 0.1112 - accuracy: 0.9543 - val_loss: 0.2577 - val_accuracy: 0.9014
Epoch 8/10
350/350 [==============================] - 28s 80ms/step - loss: 0.1029 - accuracy: 0.9621 - val_loss: 0.2319 - val_accuracy: 0.9057
Epoch 9/10
350/350 [==============================] - 28s 80ms/step - loss: 0.0670 - accuracy: 0.9746 - val_loss: 0.2834 - val_accuracy: 0.9114
Epoch 10/10
350/350 [==============================] - 28s 79ms/step - loss: 0.0807 - accuracy: 0.9721 - val_loss: 0.2466 - val_accuracy: 0.9343
Training took 292.45 seconds.
Training took 4.87 minutes.
Training took 0.08 hours.

In [ ]:
Epoch 1/10
350/350 [==============================] - 28s 80ms/step - loss: 0.0581 - accuracy: 0.9764 - val_loss: 0.3334 - val_accuracy: 0.9000
Epoch 2/10
350/350 [==============================] - 28s 80ms/step - loss: 0.0679 - accuracy: 0.9739 - val_loss: 0.3008 - val_accuracy: 0.9214
Epoch 3/10
350/350 [==============================] - 29s 82ms/step - loss: 0.0325 - accuracy: 0.9886 - val_loss: 0.2559 - val_accuracy: 0.9357
Epoch 4/10
350/350 [==============================] - 28s 80ms/step - loss: 0.0234 - accuracy: 0.9911 - val_loss: 0.2819 - val_accuracy: 0.9200
Epoch 5/10
350/350 [==============================] - 28s 80ms/step - loss: 0.0094 - accuracy: 0.9971 - val_loss: 0.3503 - val_accuracy: 0.9200
Epoch 6/10
350/350 [==============================] - 28s 81ms/step - loss: 0.0379 - accuracy: 0.9825 - val_loss: 0.3773 - val_accuracy: 0.9014
Epoch 7/10
350/350 [==============================] - 29s 82ms/step - loss: 0.1056 - accuracy: 0.9654 - val_loss: 0.3767 - val_accuracy: 0.9200
Epoch 8/10
350/350 [==============================] - 28s 80ms/step - loss: 0.0331 - accuracy: 0.9882 - val_loss: 0.3546 - val_accuracy: 0.9171
Epoch 9/10
350/350 [==============================] - 28s 80ms/step - loss: 0.0061 - accuracy: 0.9989 - val_loss: 0.3178 - val_accuracy: 0.9314
Epoch 10/10
350/350 [==============================] - 28s 79ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.3326 - val_accuracy: 0.9271
Training took 280.9 seconds.
Training took 4.68 minutes.
Training took 0.08 hours.